In [1]:
import matplotlib.pyplot as plt

import sinar_ia as SIA
from pathlib import Path
import numpy as np
from skimage.util import img_as_float, img_as_uint

plt.figure(figsize=(40, 40))

<Figure size 2880x2880 with 0 Axes>

<Figure size 2880x2880 with 0 Axes>

In [2]:
h = 5344
w = 4008
flats = Path("/Users/mgolub2/Pictures/emotion22_80mm_f4_iso50_flats/").glob("*.IA")
imgs = []
for f in flats:
    imgs.append(SIA.read_sinar(f))

In [3]:
flats = list(sorted(imgs, key=lambda x: x.shutter_count, reverse=True))[0:25]

In [ ]:
def create_master_flat(flats, h=5344, w=4008):
    corrected = np.zeros((h, w))
    num_flats = 0
    for i in flats:
        if abs(i.measured_shutter_us - i.req_shutter_us) / i.req_shutter_us > 0.5:
            # Our shutter was more than 50% slower than requested, skip this file
            print(
                f"Skipping {i.filename}, shutter speed was {i.measured_shutter_us}uS, requested {i.req_shutter_us}uS!"
            )
            continue
        nd_img = img_as_float(SIA.get_raw_pillow(i, h, w))
        black_path = i.filename.parent.absolute() / Path(i.black_ref).name
        b0, b1 = SIA.read_black_ref(black_path, nd_img)
        corrected += nd_img - b1
        num_flats += 1
    corrected /= num_flats
    return corrected


def process_dir_to_flats(flats, output_dir=".", h=5344, w=4008):
    # flats = list(directory.glob('*.IA'))
    f0: SIA.SinarIA = flats[0]
    lens = (
        f0.focal_length if f0.focal_length else input("Please enter the focal length: ")
    )
    corrected = create_master_flat(flats, h, w)
    np.save(Path(output_dir) / f"pyEMDNG_flat_iso{f0.iso}_{lens}mm", corrected)


# flat = create_master_flat(flats)
# imshow(flat, cmap='gist_yarg')
process_dir_to_flats(flats)

In [ ]:
img_path = Path("/Users/mgolub2/Pictures/002000E8.EMO/62A4DF68.IA")
img = SIA.read_sinar(img_path)

In [ ]:
nd_img = SIA.process_raw(img, h, w)

In [ ]:
black_path = img.filename.parent.absolute() / Path(img.black_ref).name
b0, b1 = SIA.read_black_ref(black_path, nd_img)

In [ ]:
plt.imshow(b0 - b1)

In [ ]:
plt.imshow(b1, cmap="gist_yarg")

In [ ]:
b1.max()

In [ ]:
b0.max()

In [ ]:
b0.min()

In [ ]:
b1.max()

In [ ]:
(b0 - b1).max()

In [ ]:
b0.max() / b1.max()